In [125]:
import numpy as np
import signal
import time
import pickle
import sys
import matplotlib.pyplot as plt
import math

In [126]:
def foo(x, G, C):
    return (x.T @ G @ x) / 2 + C.T @ x

def fooGradient(x, G, C):
    return G @ x + C

def armijo(x, alpha, direction, G, C):
    f1 = foo(x, G, C)
    f2 = foo(x + alpha * direction, G, C)
    # print("x=", x, "f1=", f1, "f2=", f2, "alpha=", alpha, "direction=", direction)
    return f2 <= f1

def happy(x, next_x):
    criteria = (np.abs(x - next_x) < 1e-16)
    return np.all(criteria)

#### Test KKT-Condition

In [127]:
def projection(x, u=2, l=-2):
    return np.clip(x, l, u)


def checkKKT(x, G, C, u=2, l=-2):
    Gx_plus_C = G @ x + C
    ll = np.maximum(Gx_plus_C, 0)  # 將所有元素小於0的部分補0
    lu = np.maximum(-Gx_plus_C, 0)  # 將所有元素大於等於0的部分補0
    print("Original:", Gx_plus_C)
    print("ll:", ll)
    print("lu:", lu)
    print("x", x)
    print("lu*(u-x)", lu.T@(u-x))
    print("ll*(x-l)", ll.T@(x-l))

def main():
    rho = 0.5
    alpha_0 = 0.05
    k = 1
    x = np.array([
    [0],
    [0],
    [0],
    [0],
    [0],
    [0]
])
    G = np.array([
    [6, 0, 0, -6, -2, 1],
    [0, 2, -2, 1, 1, 4],
    [0, -2, 4, 1, -3, -6],
    [-6, 1, 1, -2, 2, -4],
    [-2, 1, -3, 2, -4, 0],
    [1, 4, -6, -4, 0, -4]
])
    C = np.array([
    [0],
    [3],
    [0],
    [0],
    [0],
    [1]
])
    while True:
        alpha = alpha_0
        gradX = fooGradient(x, G, C)
        direction = -gradX
        while not armijo(x, alpha, direction, G, C):
            alpha *= rho
        if happy(x, projection(x + alpha * direction)):
            break
        # next step
        x = projection(x + alpha * direction)
        k = k + 1
        # print(k, x, p, q, h, fooHessian(x), alpha, gradX, direction)

    # Check KKT
    checkKKT(x, G, C)
    print()
    print("k=", k)
    print(foo(x, G, C))
    return 0
temp = main()

Original: [[ 1.77635684e-15]
 [ 7.00000000e+00]
 [-1.77635684e-15]
 [-2.40000000e+01]
 [ 8.00000000e+00]
 [-2.20000000e+01]]
ll: [[1.77635684e-15]
 [7.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [8.00000000e+00]
 [0.00000000e+00]]
lu: [[0.00000000e+00]
 [0.00000000e+00]
 [1.77635684e-15]
 [2.40000000e+01]
 [0.00000000e+00]
 [2.20000000e+01]]
x [[ 1.00000000e+00]
 [-2.00000000e+00]
 [-4.74613242e-16]
 [ 2.00000000e+00]
 [-2.00000000e+00]
 [ 2.00000000e+00]]
lu*(u-x) [[3.55271368e-15]]
ll*(x-l) [[5.32907052e-15]]

k= 180
[[-63.]]
